This notebook contains the code used to extract relavent census data that was used for demographic profile analysis. 

Sources used are linked in the readme.md document.



In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows',400)


In [54]:
"""
source: https://github.com/anutkk/sphericalgeometry
"""


def polygon_area(lats, lons, algorithm = 0, radius = 6378137):
    """
    Computes area of spherical polygon, assuming spherical Earth. 
    Returns result in ratio of the sphere's area if the radius is specified.
    Otherwise, in the units of provided radius.
    lats and lons are in degrees.
    """
    from numpy import arctan2, cos, sin, sqrt, pi, power, append, diff, deg2rad
    lats = np.deg2rad(lats)
    lons = np.deg2rad(lons)

    # Line integral based on Green's Theorem, assumes spherical Earth

    #close polygon
    if lats[0]!=lats[-1]:
        lats = append(lats, lats[0])
        lons = append(lons, lons[0])

    #colatitudes relative to (0,0)
    a = sin(lats/2)**2 + cos(lats)* sin(lons/2)**2
    colat = 2*arctan2( sqrt(a), sqrt(1-a) )

    #azimuths relative to (0,0)
    az = arctan2(cos(lats) * sin(lons), sin(lats)) % (2*pi)

    # Calculate diffs
    # daz = diff(az) % (2*pi)
    daz = diff(az)
    daz = (daz + pi) % (2 * pi) - pi

    deltas=diff(colat)/2
    colat=colat[0:-1]+deltas

    # Perform integral
    integrands = (1-cos(colat)) * daz

    # Integrate 
    area = abs(sum(integrands))/(4*pi)

    area = min(area,1-area)
    if radius is not None: #return in units of radius
        return area * 4*pi*radius**2
    else: #return in ratio of sphere total area
        return area

In [55]:
"""
Read in data and normalize columns
"""

station_nta_df = pd.read_csv("station_neighborhood_assignments.csv")

econ_data = pd.read_csv('econ_2016acs5yr_nta.csv')
hous_data = pd.read_csv('hous_2016acs5yr_nta.csv')
soc_data = pd.read_csv('soc_2016acs5yr_nta.csv')
demo_data = pd.read_csv('demo_2016acs5yr_nta.csv')

econ_data['GeogName_GeoId'] = econ_data.GeogName + '_' + econ_data.GeoID

geomName_geomID = pd.DataFrame(econ_data.GeogName_GeoId.unique(), columns = ['GeogName_GeoID'])
geomName_geomID = geomName_geomID.GeogName_GeoID.str.split(pat='_', expand=True)
geomName_geomID.columns = ['NTA', 'NTA_ID']
geomName_geomID.NTA = geomName_geomID.NTA.str.upper()

"""Manual fixing string mismatch"""
geomName_geomID.iloc[61]['NTA'] = "HUDSON YARDS-CHELSEA-FLATIRON-UNION SQUARE"
geomName_geomID.iloc[21]['NTA'] = "EAST NEW YORK PENNSYLVANIA AVE"

nta_id_list = []
for row in station_nta_df.itertuples():
    nta_id = geomName_geomID.NTA_ID[geomName_geomID.NTA == row.NTA].values[0]
    nta_id_list.append([nta_id])

    
nta_id_series = pd.Series(nta_id_list).str[0]
station_nta_df['NTA_ID'] = nta_id_series
station_nta_df.columns =['Station','GeogName','GeoID']
station_nta_df = station_nta_df.sort_values(by=['GeogName'])
station_nta_df.reset_index(drop=True, inplace=True)


In [56]:
"""
Cleaning econ, housing, and social data set + keeping station NTAs
"""


station_econ_data = econ_data[econ_data.GeoID.isin(station_nta_df.GeoID)]
station_hous_data = hous_data[hous_data.GeoID.isin(station_nta_df.GeoID)]
station_soc_data = soc_data[soc_data.GeoID.isin(station_nta_df.GeoID)]
station_demo_data = demo_data[demo_data.GeoID.isin(station_nta_df.GeoID)]

station_econ_data.to_csv("station_econ_data.csv",index=False)
station_hous_data.to_csv("station_hous_data.csv",index=False)
station_soc_data.to_csv("station_soc_data.csv",index=False)
station_demo_data.to_csv("station_demo_data.csv",index=False)

In [57]:
"""
Filter all data for only manhattan stations
"""

manhattan_station_econ_data = station_econ_data[station_econ_data.Borough == 'Manhattan'].fillna(0)
manhattan_station_demo_data = station_demo_data[station_demo_data.Borough == 'Manhattan'].fillna(0)
manhattan_station_hous_data = station_hous_data[station_hous_data.Borough == 'Manhattan'].fillna(0)
manhattan_station_soc_data = station_soc_data[station_soc_data.Borough == 'Manhattan'].fillna(0)

manhattan_station_econ_data.GeogName = manhattan_station_econ_data.GeogName.str.upper()
manhattan_station_demo_data.GeogName = manhattan_station_econ_data.GeogName.str.upper()
manhattan_station_hous_data.GeogName = manhattan_station_econ_data.GeogName.str.upper()
manhattan_station_soc_data.GeogName = manhattan_station_soc_data.GeogName.str.upper()

manhattan_station_econ_data = manhattan_station_econ_data.sort_values(by=['GeogName']).reset_index(drop=True)
manhattan_station_demo_data = manhattan_station_demo_data.sort_values(by=['GeogName']).reset_index(drop=True)
manhattan_station_hous_data = manhattan_station_hous_data.sort_values(by=['GeogName']).reset_index(drop=True)
manhattan_station_soc_data = manhattan_station_soc_data.sort_values(by=['GeogName']).reset_index(drop=True)

manhattan_demographics_df = pd.DataFrame()
manhattan_demographics_df['GeogName'] = manhattan_station_econ_data.GeogName


manhattan_demographics_df['age_18_29'] = manhattan_station_demo_data.Pop20t24P + manhattan_station_demo_data.Pop25t29P
manhattan_demographics_df['age_30_49'] = manhattan_station_demo_data.Pop30t34P + manhattan_station_demo_data.Pop35t39P + manhattan_station_demo_data.Pop40t44P + manhattan_station_demo_data.Pop45t49P
manhattan_demographics_df['income_30_74'] = manhattan_station_econ_data.FamI35t49P + manhattan_station_econ_data.HHI50t74P 
manhattan_demographics_df['income_75_plus'] = manhattan_station_econ_data.FamI75t99P + manhattan_station_econ_data.HI100t149P + manhattan_station_econ_data.HI150t199P + manhattan_station_econ_data.HHI200plP
manhattan_demographics_df['vehicles_none'] = manhattan_station_hous_data.NoVhclAvP
manhattan_demographics_df['vehicles_one'] = manhattan_station_hous_data.Vhcl1AvP
manhattan_demographics_df['edu_HS_less'] = manhattan_station_soc_data.EA_LT9GP + manhattan_station_soc_data.EA_9t12NDP + manhattan_station_soc_data.EA_HScGrdP
manhattan_demographics_df['edu_some_college'] = manhattan_station_soc_data.EA_SClgNDP
manhattan_demographics_df['edu_college_plus'] = manhattan_station_soc_data.EA_AscDP + manhattan_station_soc_data.EA_BchDP + manhattan_station_soc_data.EA_GrdPfDP
manhattan_demographics_df = manhattan_demographics_df.reset_index(drop=True)


In [58]:
"""
Filter manhattan data for neighborhoods with subway stations
"""

manhattan_neighborhoods = pd.DataFrame(manhattan_station_econ_data.GeogName.unique(),columns=['GeogName'])
manhattan_neighborhoods.GeogName = manhattan_neighborhoods.GeogName.str.upper()
manhattan_neighborhoods['GeogName'][3] = 'HUDSON YARDS-CHELSEA-FLATIRON-UNION SQUARE'
manhattan_station_nta = station_nta_df[station_nta_df.GeogName.isin(manhattan_neighborhoods.GeogName)]
manhattan_station_nta.to_csv("manhattan_station_nta.csv",index=False)



In [59]:
"""
Read in city polygons and filter for manhattan polygons
source file: https://data.cityofnewyork.us/City-Government/NTA-map/d3qk-pfyz
borough_nta_polygons.csv: attached; consists of NTA, Borough, polygon coordinates from source file
"""
polygons = pd.read_csv("borough_nta_polygons.csv")
polygons.NTA = polygons.NTA.str.replace(',',' ',regex=True).str.upper()
manhattan_polygons = polygons[polygons.Borough == 'Manhattan']

In [60]:
"""
For manhattan polygons, re-format lat and long array and use to calculate square footage
"""

sq_ft_list = []

for row in polygons.itertuples():
    lats = []
    longs = []
    for index, element in enumerate(row.geom.split(',')):
        if index%2 == 0:
            lats.append(element)
        else:
            longs.append(element)
    lats_arr = np.array(lats).astype(float)
    longs_arr = np.array(longs).astype(float)
    
    sq_ft = polygon_area(lats_arr, longs_arr, algorithm = 0, radius = 6378137)
    sq_ft_list.append(sq_ft)
    
sq_ft_series = pd.Series(sq_ft_list)

polygons['square_footage'] = sq_ft_series
manhattan_nta_sq_footage = polygons[polygons.Borough == 'Manhattan']
manhattan_nta_sq_footage = manhattan_nta_sq_footage.sort_values(by=['NTA']).reset_index(drop=True)
#manhattan_nta_sq_footage.to_csv('manhattan_nta_sq_footage.csv',index=False)

In [61]:
mta_manhattan_nta

,GeogName,GeoID
0,CHINATOWN,MN27
1,EAST VILLAGE,MN22
2,GRAMERCY,MN21
3,HUDSON YARDS-CHELSEA-FLATIRON-UNION SQUARE,MN13
4,LENOX HILL-ROOSEVELT ISLAND,MN31
5,LINCOLN SQUARE,MN14
6,LOWER EAST SIDE,MN28
7,MANHATTANVILLE,MN06
8,MARBLE HILL-INWOOD,MN01
9,MIDTOWN-MIDTOWN SOUTH,MN17


In [62]:
"""
Calculate square footage for each manhattan NTA with a subway station
"""

mta_manhattan_nta = manhattan_station_nta[['GeogName','GeoID']].drop_duplicates().reset_index(drop=True)

sq_ft_list = []
for row in mta_manhattan_nta.itertuples():
    sq_ft = manhattan_nta_sq_footage.square_footage[manhattan_nta_sq_footage.NTA == row.GeogName].values[0]
    sq_ft_list.append(sq_ft)

sq_ft_series = pd.DataFrame(sq_ft_list)

mta_manhattan_nta_sq_foot = mta_manhattan_nta.reset_index(drop=True)
mta_manhattan_nta_sq_foot['square_ft'] = sq_ft_series

In [63]:
mta_manhattan_nta_sq_foot

,GeogName,GeoID,square_ft
0,CHINATOWN,MN27,4.906073e+05
1,EAST VILLAGE,MN22,3.688185e+05
2,GRAMERCY,MN21,2.548831e+05
3,HUDSON YARDS-CHELSEA-FLATIRON-UNION SQUARE,MN13,1.254305e+06
4,LENOX HILL-ROOSEVELT ISLAND,MN31,2.820291e+05
5,LINCOLN SQUARE,MN14,5.355045e+05
6,LOWER EAST SIDE,MN28,4.306251e+05
7,MANHATTANVILLE,MN06,3.616412e+05
8,MARBLE HILL-INWOOD,MN01,6.848097e+05
9,MIDTOWN-MIDTOWN SOUTH,MN17,1.022698e+06


In [64]:
"""
Read in total housing dataset and filter for Manhattan NTA with subway stations
"""

total_housing = pd.read_csv("tothousing_vacant_2000-2010nta.csv")
total_housing.columns = total_housing.iloc[3]
mta_manhattan_total_housing = total_housing[total_housing['Neighborhood Tabulation Area (NTA)*'].isin(mta_manhattan_nta_sq_foot.GeoID)]
mta_manhattan_total_housing.columns = ['Borough',
                                      '2010_Census_County_Code',
                                      'NTA_ID',
                                      'NTA',
                                      '2000_Total_Housing_Units',
                                      '2000_Occupied_Housing_Units',
                                      '2000_Vacant_Housing_Units',
                                       '2010_Total_Housing_Units',
                                      '2010_Occupied_Housing_Units',
                                      '2010_Vacant_Housing_Units',
                                       '%Change_Total_Housing_Units',
                                      '%Change_Occupied_Housing_Units',
                                      '%Change_Vacant_Housing_Units',
                                       'Nan','Nan','Nan','Nan'
                                      ]
mta_manhattan_total_housing.NTA = mta_manhattan_total_housing.NTA.str.upper()
mta_manhattan_total_housing = mta_manhattan_total_housing.sort_values(by=['NTA']).reset_index(drop=True)

/Users/kaitlin/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [65]:
mta_manhattan_total_housing

,Borough,2010_Census_County_Code,NTA_ID,NTA,2000_Total_Housing_Units,2000_Occupied_Housing_Units,2000_Vacant_Housing_Units,2010_Total_Housing_Units,2010_Occupied_Housing_Units,2010_Vacant_Housing_Units,%Change_Total_Housing_Units,%Change_Occupied_Housing_Units,%Change_Vacant_Housing_Units,Nan,Nan,Nan,Nan
0,Manhattan,061,MN27,CHINATOWN,"18,642","17,949",693,"19,565","18,494","1,071",923,5.0,545,3.0,378,54.5,NaN
1,Manhattan,061,MN22,EAST VILLAGE,"24,061","23,024","1,037","24,653","23,116","1,537",592,2.5,92,0.4,500,48.2,NaN
2,Manhattan,061,MN21,GRAMERCY,"17,518","16,548",970,"17,194","15,855","1,339",-324,-1.8,-693,-4.2,369,38.0,NaN
3,Manhattan,061,MN13,HUDSON YARDS-CHELSEA-FLAT IRON-UNION SQUARE,"34,775","32,386","2,389","46,775","40,887","5,888","12,000",34.5,"8,501",26.2,3499,146.5,NaN
4,Manhattan,061,MN31,LENOX HILL-ROOSEVELT ISLAND,"49,736","45,762","3,974","51,419","45,312","6,107","1,683",3.4,-450,-1.0,2133,53.7,NaN
5,Manhattan,061,MN14,LINCOLN SQUARE,"36,433","33,062","3,371","39,492","34,216","5,276","3,059",8.4,"1,154",3.5,1905,56.5,NaN
6,Manhattan,061,MN28,LOWER EAST SIDE,"30,068","28,660","1,408","31,846","30,578","1,268","1,778",5.9,"1,918",6.7,-140,-9.9,NaN
7,Manhattan,061,MN06,MANHATTANVILLE,"8,754","8,240",514,"8,558","7,991",567,-196,-2.2,-249,-3.0,53,10.3,NaN
8,Manhattan,061,MN01,MARBLE HILL-INWOOD,"18,396","17,920",476,"18,903","18,129",774,507,2.8,209,1.2,298,62.6,NaN
9,Manhattan,061,MN17,MIDTOWN-MIDTOWN SOUTH,"18,890","15,302","3,588","22,080","16,940","5,140","3,190",16.9,"1,638",10.7,1552,43.3,NaN


In [66]:
"""
Manual Fix for string match
"""
mta_manhattan_nta_sq_foot.GeogName[3] = 'HUDSON YARDS-CHELSEA-FLAT IRON-UNION SQUARE'

/Users/kaitlin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [67]:
"""
Calculate occupied housing density as occupied housing units per square mile
"""

sq_ft_list = []

for row in mta_manhattan_total_housing.itertuples():
    sq_ft = mta_manhattan_nta_sq_foot.square_ft[mta_manhattan_nta_sq_foot.GeoID == row[3]].values[0]
    sq_ft_list.append(sq_ft)

sq_ft_series = pd.DataFrame(sq_ft_list)
mta_manhattan_total_housing['square_ft'] = sq_ft_series

mta_manhattan_total_housing = mta_manhattan_total_housing.iloc[:,[0,2,3,7,8,9,17]]
mta_manhattan_total_housing.columns = ['Borough','NTA_ID','NTA','total_housing_units','occupied_housing_units','vacant_housing_units','square_ft']
mta_manhattan_total_housing.total_housing_units = mta_manhattan_total_housing.total_housing_units.str.replace(',','',regex=True)
mta_manhattan_total_housing.occupied_housing_units = mta_manhattan_total_housing.occupied_housing_units.str.replace(',','',regex=True)

mta_manhattan_total_housing['square_mi'] = mta_manhattan_total_housing.square_ft * 3.58701e-8
mta_manhattan_total_housing['total_housing_density_mi'] = mta_manhattan_total_housing.total_housing_units.astype(float) / mta_manhattan_total_housing.square_mi
mta_manhattan_total_housing['occupied_housing_density_mi'] = mta_manhattan_total_housing.occupied_housing_units.astype(float) / mta_manhattan_total_housing.square_mi

/Users/kaitlin/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/kaitlin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/kaitlin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [68]:
"""
Classifying percent housing:
    Low density = 1 unit, 2 unit, 3 or 4 unit, Mobile home, Boat, RV, van, etc
    Medium density = 5-9 unit, 10-19 unit
    High density = 20 or more unit
"""

mta_manhattan_total_housing['ratio_low_density'] = (manhattan_station_hous_data.HU1uDP + manhattan_station_hous_data.HU1uAP + manhattan_station_hous_data.HU2uP + manhattan_station_hous_data.HU3t4uP + manhattan_station_hous_data.MobHmP + manhattan_station_hous_data.BtRVVEtcP)/100
mta_manhattan_total_housing['ratio_med_density'] = (manhattan_station_hous_data.HU5t9uP + manhattan_station_hous_data.HU10t19uP)/100
mta_manhattan_total_housing['ratio_high_density'] = (manhattan_station_hous_data.HU20pluP)/100

## What percent of the density is low,med,high
mta_manhattan_total_housing['rel_occ_low_density'] = mta_manhattan_total_housing.ratio_low_density * mta_manhattan_total_housing.occupied_housing_density_mi
mta_manhattan_total_housing['rel_occ_med_density'] = mta_manhattan_total_housing.ratio_med_density * mta_manhattan_total_housing.occupied_housing_density_mi
mta_manhattan_total_housing['rel_occ_high_density'] = mta_manhattan_total_housing.ratio_high_density * mta_manhattan_total_housing.occupied_housing_density_mi



/Users/kaitlin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/kaitlin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/kaitlin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [69]:
mta_manhattan_total_housing

,Borough,NTA_ID,NTA,total_housing_units,occupied_housing_units,vacant_housing_units,square_ft,square_mi,total_housing_density_mi,occupied_housing_density_mi,ratio_low_density,ratio_med_density,ratio_high_density,rel_occ_low_density,rel_occ_med_density,rel_occ_high_density
0,Manhattan,MN27,CHINATOWN,19565,18494,"1,071",4.906073e+05,0.017598,1.111766e+06,1.050907e+06,0.056,0.351,0.594,58850.790772,368868.349306,6.242387e+05
1,Manhattan,MN22,EAST VILLAGE,24653,23116,"1,537",3.688185e+05,0.013230,1.863479e+06,1.747300e+06,0.053,0.374,0.574,92606.889134,653490.123321,1.002950e+06
2,Manhattan,MN21,GRAMERCY,17194,15855,"1,339",2.548831e+05,0.009143,1.880630e+06,1.734174e+06,0.044,0.114,0.841,76303.655904,197695.835751,1.458440e+06
3,Manhattan,MN13,HUDSON YARDS-CHELSEA-FLAT IRON-UNION SQUARE,46775,40887,"5,888",1.254305e+06,0.044992,1.039628e+06,9.087608e+05,0.057,0.162,0.781,51799.366439,147219.251983,7.097422e+05
4,Manhattan,MN31,LENOX HILL-ROOSEVELT ISLAND,51419,45312,"6,107",2.820291e+05,0.010116,5.082731e+06,4.479058e+06,0.030,0.140,0.829,134371.747187,627068.153539,3.713139e+06
5,Manhattan,MN14,LINCOLN SQUARE,39492,34216,"5,276",5.355045e+05,0.019209,2.055954e+06,1.781286e+06,0.033,0.090,0.876,58782.424011,160315.701849,1.560406e+06
6,Manhattan,MN28,LOWER EAST SIDE,31846,30578,"1,268",4.306251e+05,0.015447,2.061688e+06,1.979598e+06,0.030,0.136,0.836,59387.954701,269225.394647,1.654944e+06
7,Manhattan,MN06,MANHATTANVILLE,8558,7991,567,3.616412e+05,0.012972,6.597232e+05,6.160141e+05,0.032,0.147,0.821,19712.449778,90554.066170,5.057475e+05
8,Manhattan,MN01,MARBLE HILL-INWOOD,18903,18129,774,6.848097e+05,0.024564,7.695348e+05,7.380255e+05,0.040,0.075,0.885,29521.020977,55351.914331,6.531526e+05
9,Manhattan,MN17,MIDTOWN-MIDTOWN SOUTH,22080,16940,"5,140",1.022698e+06,0.036684,6.018925e+05,4.617780e+05,0.031,0.082,0.886,14315.118600,37865.797588,4.091353e+05


In [97]:
"""
Filter and export data
"""
top_10_df = pd.read_csv("top_10_nta_changes.csv")
top_10 = top_10_df.GeogName
top_10[4] = 'HUDSON YARDS-CHELSEA-FLAT IRON-UNION SQUARE'

top_10_housing = mta_manhattan_total_housing[mta_manhattan_total_housing.NTA.isin(top_10)]
top_10_housing = top_10_housing.sort_values(by=['NTA']).reset_index(drop=True)

top_10_demographics = manhattan_demographics_df[manhattan_demographics_df.GeogName.isin(top_10)]
top_10_demographics = top_10_demographics.sort_values(by=['GeogName']).reset_index(drop=True)

top_10_demographics.to_csv("top_10_manhattan_demographics.csv", index=False)
top_10_housing.to_csv("top_10_manhattan_housing_densities.csv",index=False)

""""""

top_5 = top_10[0:5]
top_5_demographics = manhattan_demographics_df[manhattan_demographics_df.GeogName.isin(top_5)]
top_5_housing = mta_manhattan_total_housing[mta_manhattan_total_housing.NTA.isin(top_5)]

top_5_demographics.to_csv("top_5_manhattan_demographics.csv",index=False)
top_5_housing.to_csv("top_5_manhattan_housing_densities.csv",index=False)

""""""

mta_manhattan_total_housing.to_csv("manhattan_housing_densities.csv",index=False)
manhattan_demographics_df.to_csv("manhattan_demographics_data.csv",index=False)

/Users/kaitlin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [101]:
top_5_housing

,Borough,NTA_ID,NTA,total_housing_units,occupied_housing_units,vacant_housing_units,square_ft,square_mi,total_housing_density_mi,occupied_housing_density_mi,ratio_low_density,ratio_med_density,ratio_high_density,rel_occ_low_density,rel_occ_med_density,rel_occ_high_density
3,Manhattan,MN13,HUDSON YARDS-CHELSEA-FLAT IRON-UNION SQUARE,46775,40887,"5,888",1.254305e+06,0.044992,1.039628e+06,9.087608e+05,0.057,0.162,0.781,51799.366439,147219.251983,7.097422e+05
5,Manhattan,MN14,LINCOLN SQUARE,39492,34216,"5,276",5.355045e+05,0.019209,2.055954e+06,1.781286e+06,0.033,0.090,0.876,58782.424011,160315.701849,1.560406e+06
12,Manhattan,MN24,SOHO-TRIBECA-CIVIC CENTER-LITTLE ITALY,23081,20923,"2,158",8.464123e+05,0.030361,7.602214e+05,6.891431e+05,0.094,0.299,0.607,64779.449257,206053.780082,4.183098e+05
13,Manhattan,MN19,TURTLE BAY-EAST MIDTOWN,37427,31555,"5,872",5.895396e+05,0.021147,1.769862e+06,1.492185e+06,0.026,0.068,0.906,38796.805316,101468.567750,1.351919e+06
17,Manhattan,MN23,WEST VILLAGE,42810,38582,"4,228",8.446324e+05,0.030297,1.413009e+06,1.273457e+06,0.094,0.203,0.702,119705.000134,258511.861992,8.939671e+05
